In [213]:
import re
import pprint
from IPython.display import display
from colorama import Fore, Back, Style

In [2]:
service_words = ['program', 'var', 'const', 'integer', 'real', 'string', 'label',
                 'array', 'of', 'procedure', 'function',
                 'begin', ':=', 'goto', 'if', 'then', 'else', 'end', 'end.']
operations = ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>=']
separators = [' ', ',', '..', ':', ';', '(', ')', '[', ']', '{', '}', '\'']

In [173]:
def filter_program(text):
    splitted_text = text.split('\'')
    formatted_text = []
    even_flag = False
    for span in splitted_text:
        if not even_flag:
            formatted_span = re.sub(r'[\t\n]+', ' ', span)
            formatted_span = re.sub(r' +', ' ', formatted_span)
            formatted_span = re.sub(r'\{[.\n]*\}+', '', formatted_span)
            formatted_text.append({
                'text': formatted_span.lower(),
                'type': 'code'
            })
        else:
            formatted_text.append({
                'text': span,
                'type': 'string'
            })

        even_flag = not even_flag
    if not even_flag:
        print(Fore.RED + 'Unexpected end of file. There is unclosed apostrophe!' + Style.RESET_ALL)
        return None
    
#     formatted_text = formatted_text.replace('\\', '\\\\')

    return formatted_text
        

In [174]:
class Analyzer:
    state = 'S'
    string = ''
    collecting_string = ''
    
    def __init__(self, string):
        self.string = string
        
    def reset(self):
        self.collecting_string = ''
        self.state = 'S'
        
    def unexpected(self, symbol):
        self.state = 'error'
        print(Back.RED, Fore.WHITE,'ERRORE!', Style.RESET_ALL, 'Unexpected symbol \"' + symbol + '\"')
        return {
            'kind': 'error',
            'token': symbol,
            'residue': ''
        }
    
    def symbol_return(self, symbol):
        self.string = symbol + self.string
        if symbol != '':
            self.collecting_string = self.collecting_string[:-1]
    
    def collect_next(self):
#         print('String: "',  end='')
#         print(Fore.BLUE + self.string + Style.RESET_ALL, end='')
#         print('"')
        
        try:
            symbol = self.string[0]
            self.string = self.string[1:]
        except:
            symbol = ''
        self.collecting_string += symbol
        
        if self.state == 'S':
            if symbol.isalpha() or symbol == '_':
                self.state = 'letter_at_first'
            elif symbol == '<':
                self.state = '<_at_first'
            elif symbol == '>':
                self.state = '>_at_first'
            elif symbol in operations:
                return {
                    'kind': 'operation',
                    'token': symbol,
                    'residue': self.string
                }
            elif symbol.isdigit():
                self.state = 'digit_at_first'
            elif symbol == '.':
                self.state = '._at_first'
            return self.collect_next()
        
        if self.state in ['number -> .. -> number']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'integer_interval',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
        
        if self.state in ['number -> ..']:
            if symbol.isdigit():
                self.state = 'number -> .. -> number'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
        
        if self.state == 'digit_at_first':
            if symbol.isdigit():
                return self.collect_next()
            elif symbol == '.':
                self.state = 'number -> .'
                return self.collect_next()
            elif symbol == 'e':
                self.state = 'number -> e'
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'integer',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
            
        if self.state == 'number -> .':
            if symbol == '.':
                self.symbol_return(symbol)
                self.symbol_return(symbol)
                return {
                    'kind': 'integer',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            
        if self.state == '._at_first':
            if symbol == '.':
                return {
                    'kind': 'separator',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state in ['._at_first', 'number -> .']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol == 'e':
                self.state = 'number -> e'
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'real',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            elif symbol == '.':
                self.state = 'number -> ..'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
                
        if self.state == 'number -> e':
            if symbol in ['+', '-']:
                self.state = 'number -> e -> +/-'
                return self.collect_next()
            elif symbol.isdigit():
                self.state = 'number -> e -> digit'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
        
        if self.state in ['number -> e -> +/-', 'number -> e -> digit']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'real',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
                
        if self.state == '<_at_first':
            if symbol in ['>', '=']:
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                self.symbol_return(symbol)
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state == '>_at_first':
            if symbol == '=':
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                self.symbol_return(symbol)
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state == 'letter_at_first':
            if symbol.isalpha() or symbol.isdigit() or symbol == '_':
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'identifier',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
            
        return {
            'kind': 'exeption',
            'token': str(self.state),
            'residue': self.string
        }

In [175]:
def split_by_separator(line):
    global separators
    
#     Search first separator or assignment
    first_separator_index = len(line)
    for character in line:
        if character in separators:
            first_separator_index = line.index(character)
            break
    
#   Token kind determinination
    token = line[:first_separator_index]
    separator = ''
    if first_separator_index < len(line):
        separator = line[first_separator_index]
    
    kind = 'unknown'
    if token in service_words:
        kind = 'service_word'
    elif token in operations:
        kind = 'operation'
        
#     Checking for String
    elif not token and separator == '\'':
        kind = 'string'
        
#         Search second apostrophe
        second_apostrophe_index = 1
        if len(line) > 1:
            while line[second_apostrophe_index] != '\'' and second_apostrophe_index < len(line):
                second_apostrophe_index+=1
                
#         Slicing string constant from line
        token = line[1:second_apostrophe_index]
        separator = '\''
        first_separator_index = second_apostrophe_index-1
        
        return {
            'token': token,
            'kind': kind,
            'separator': separator,
            'remaining_line': line[second_apostrophe_index+1:]
        }
#     Assignment regognition
    elif not token and separator == ':' and line[first_separator_index+1] == '=':
        kind = 'service_word'
        token = ':='
        separator = ''
        first_separator_index+=1
        
    else:
#         Checking for Int, Real, Indetifier
        try:
            int(token)
            kind = 'integer'
        except:
            kind = 'unknown'
    
#     Preparation to detect assignment operator
    if separator == ':':
        try:
            if line[first_separator_index+1] == '=':
                separator = ''
                first_separator_index-=1
        except:
            pass
        
    if separator == '\'':
        first_separator_index-=1
    
    try:
        if line[first_separator_index+1] == '\'' and (not separator.isspace()) and separator != '\'':
            line = line[:first_separator_index+1] + ' ' + line[first_separator_index+1:]
        elif line[first_separator_index+1] == '\'' and separator.isspace():
            separator = '\'';
    except:
        pass

    return {
        'token': token,
        'kind': kind,
        'separator': separator,
        'remaining_line': line[first_separator_index+1:]
    }

In [176]:
def append_consumable(token, kind, number_of_procedure=0, level_of_procedure=0, number_in_procedure=0):
    global constants
    global identifiers
    
    if kind in ['integer', 'real', 'string', 'integer_interval']:
        constant = {
            'type': kind,
            'value': token
        }
        if not constant in constants:
            constants.append(constant)
    elif kind == 'identifier':
        identifier = {
            'type': kind,
            'name': token,
            'number_of_procedure': number_of_procedure,
            'level_of_procedure': level_of_procedure,
            'number_in_procedure': number_in_procedure
        }
        if not identifier in identifiers:
            identifiers.append(identifier)

In [177]:
def to_token_analyze(file_name):
    global tokens_chain
    
    program = filter_program(open(file_name).read())
    print(program)
    
    if not program:
        return None
    
#     print(Fore.BLUE, program, Style.RESET_ALL)

    tokens_chain = []
    
    program_context_flag = False
    var_context_flag = False
    array_context_flag = False
    label_context_flag = False
    function_context_flag = False
    procedure_context_flag = False
    var_pool = []

    for segment in program:
        if segment['type'] == 'code':
#             for line in lines:
            remaining_line = segment['text']
            splitted_line = split_by_separator(remaining_line)
            while remaining_line:
                token = splitted_line['token']
                kind = splitted_line['kind']
                separator = splitted_line['separator']
                remaining_line = splitted_line['remaining_line']

                kind_suffix = ''
                if token or kind == 'string':                
                    if kind == 'service_word':                        
                        tokens_chain.append([
                            'W',
                            service_words.index(token),
                            token
                        ])
                        kind_suffix = 'W' + str(service_words.index(token))

                        if token == 'array':
                            array_context_flag = True
                        else:
                            if token in ['integer', 'real', 'string']:
                                var_type = token
                                if array_context_flag:
                                    var_type += '_array';

                                for var in var_pool:
                                    identifiers[:] = [d for d in identifiers if d.get('name') != var]
                                    identifiers.append({
                                        'type': var_type,
                                        'name': var,
                                        'number_of_procedure': 0,
                                        'level_of_procedure': 0,
                                        'number_in_procedure': 0
                                    })
                                var_pool = []
                                array_context_flag = False

                            elif token != 'of':
                                if token == 'var':
                                    var_context_flag = True
                                else:
                                    var_context_flag = False

                                if token == 'program':
                                    program_context_flag = True
                                else:
                                    program_context_flag = False

                                if token == 'label':
                                    label_context_flag = True
                                else:
                                    label_context_flag = False

                                if token == 'function':
                                    function_context_flag = True
                                else:
    #                                 print('FUNCTION FLAG FALSE')
                                    function_context_flag = False


                    elif kind == 'operation':
                        tokens_chain.append([
                            'O',
                            operations.index(token),
                            token
                        ])
                        kind_suffix = 'O' + str(operations.index(token))

                    if kind == 'unknown':

    #                     Automat analyze

                        automat = Analyzer(token)
                        residue = token

                        while residue:
                            automat.reset()
                            analyzed = automat.collect_next()
                            kind = analyzed['kind']
                            token = analyzed['token']
                            residue = analyzed['residue']

                            if kind == 'exeption':
                                print(Fore.BLACK + '\"' + token + '\"', Fore.RED + kind, Fore.WHITE + 'by analyzer' + Style.RESET_ALL, residue)
                            else:

    #                             Var pool
                                if kind == 'identifier' and (var_context_flag or program_context_flag or label_context_flag):
                                    var_pool.append(token)

    #                             New identifier
                                if kind == 'identifier' and not len([d for d in identifiers if d.get('name') == token]):
                                    append_consumable(token, kind)                                


    #                             Constant
                                if kind in ['integer', 'real', 'string']:
                                    append_consumable(token, kind)
                                    for index, constant in enumerate(constants):
                                        if constant['value'] == token:
                                            break
                                    tokens_chain.append([
                                        'C',
                                        index,
                                        constant['value']
                                    ])
                                    kind_suffix = 'C' + str(index)

    #                             Identifier
                                elif kind == 'identifier':
                                    for index, identifier in enumerate(identifiers):
                                        if identifier['name'] == token:
                                            break
                                    tokens_chain.append([
                                        'I',
                                        index,
                                        identifier['name']
                                    ])
                                    kind_suffix = 'I' + str(index)

    #                             Separator
                                elif kind in ['separator']:
                                    tokens_chain.append([
                                        'R',
                                        separators.index(token),
                                        token
                                    ])
                                    kind_suffix = 'R' + str(separators.index(token))

    #                             Operation
                                elif kind in ['operation']:
                                    tokens_chain.append([
                                        'O',
                                        operations.index(token),
                                        token
                                    ])
                                    kind_suffix = 'O' + str(operations.index(token))
                                print(Fore.BLACK + '\"' + token + '\"', Fore.GREEN + kind, Fore.WHITE + 'by analyzer' + Style.RESET_ALL, kind_suffix)

                    else:
    #                     Pre-recognized constants or handled service words
                        if kind in ['integer', 'string']:
                            append_consumable(token, kind)
                            for index, constant in enumerate(constants):
                                if constant['value'] == token:
                                    break
                            tokens_chain.append([
                                'C',
                                index,
                                constant['value']
                            ])
                            kind_suffix = 'C' + str(index)
                        print(Fore.BLACK + '\"' + token + '\"', Fore.GREEN + kind, Style.RESET_ALL + kind_suffix)

                if not separator.isspace():
                    if separator == ';':
                        if program_context_flag:
                            var_type = 'program'
                        else:
                            var_type = 'identifier'
                        if array_context_flag:
                            var_type += '_array';

                        for var in var_pool:
                            identifiers[:] = [d for d in identifiers if d.get('name') != var]
                            identifiers.append({
                                'type': var_type,
                                'name': var,
                                'number_of_procedure': 0,
                                'level_of_procedure': 0,
                                'number_in_procedure': 0
                            })
                        var_pool = []
                            
                    if separator in separators and separator != ' ':
                        tokens_chain.append([
                            'R',
                            separators.index(separator),
                            separator
                        ])
                        print(Fore.BLACK + '\"' + separator + '\"', Fore.GREEN + 'separator', Style.RESET_ALL + 'R' + str(separators.index(separator)))
                        if separator == ';':
                            print()

                splitted_line = split_by_separator(remaining_line)
        else:
#             String handle
            append_consumable(segment['text'], 'string')
            for index, constant in enumerate(constants):
                if constant['value'] == segment['text']:
                    break
            tokens_chain.append([
                'C',
                index,
                constant['value']
            ])
            kind_suffix = 'C' + str(index)
            print(Fore.BLACK + '\"' + segment['text'] + '\"', Fore.RED + 'STRING', Fore.WHITE + 'by formatter' + Style.RESET_ALL)
            
    return tokens_chain

In [246]:
constants = []
identifiers = []    

# token_chain = to_token_analyze('./example.pas')
# token_chain = to_token_analyze('./simple.pas')
tokens_chain = to_token_analyze('./nk.pas')
# token_chain = to_token_analyze('./denis.pas')
# token_chain = to_token_analyze('./apostrophe.pas')
print()

if tokens_chain:
    for word in tokens_chain:
#         print(word[0] + str(word[1]), end=' ')
        print(word, end=' ')
    # print(tokens_chain)
    print()
    print()

    for i in identifiers:
        print(Fore.RED + i['name'], Fore.GREEN + i['type'])
    print()

    for const in constants:
        if const['type'] == 'string':
            print(Fore.BLUE + '\'' + const['value'] + '\'', Fore.GREEN + const['type'])
        else:
            print(Fore.BLUE + const['value'], Fore.GREEN + const['type'])
    print(Style.RESET_ALL, end='')
else:
    print(tokens_chain)

[{'text': 'program testovaya; var b,c,d:integer; a, e,f,g:real; s1, s2, s3,s4:string; super_array: array [1..3, 10..20] of integer; z: real; kk: array [1..2] of string; label m; const pi = 3.14; alpha = 332.1; betta = 4; begin a := 5e+11; a := 5.e+11; a := .5e+11; a := 5e11; a := 5e-11; a := 5.234e11; a := 5.e-11; a := .5e11; a := .5; a := 2.5; a := 5.; a:=5^4; if a>=34 then begin if a<=20 then if a=10 then a:=1 else a:=2; end; if a<>32 then goto m; a:=31; m: a:=32; end.', 'type': 'code'}]
"program" service_word W0
"testovaya" identifier by analyzer I0
";" separator R4

"var" service_word W1
"b" identifier by analyzer I1
"," separator R1
"c" identifier by analyzer I2
"," separator R1
"d" identifier by analyzer I3
":" separator R3
"integer" service_word W3
";" separator R4

"a" identifier by analyzer I4
"," separator R1
"e" identifier by analyzer I5
"," separator R1
"f" identifier by analyzer I6
"," separator R1
"g" identifier by analyzer I7
":" separator R3
"real" service_word W4
";" s

In [267]:
class BasicGenerator:
    program_on_basic = ''
    tokens_chain = []
    
    state = 'S'
    
    pascal_tables = {
        'W': ['program', 'var', 'const', 'integer', 'real', 'string', 'label',
                 'array', 'of', 'procedure', 'function',
                 'begin', ':=', 'goto', 'if', 'then', 'else', 'end', 'end.'],
        'O': ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>='],
        'R': [' ', ',', '..', ':', ';', '(', ')', '[', ']', '{', '}', '\''],
    }
    basic_tables = {
        'W': ['', 'var', 'const', 'integer', 'real', 'string', 'label',
                     'array', 'of', 'GOSUB', 'GOSUB',
                     'begin', '=', 'goto', 'IF', 'THEN', 'ELSE', 'END', 'END'],
        'O': ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>='],
        'R': [' ', ',', '..', ':', '\n', '(', ')', '[', ']', '{', '}', '\''],
    }
    
    var_pool = []
    array_interval = ''
    
    def __init__(self, tokens_chain, tables):
        self.tokens_chain = tokens_chain
        self.pascal_tables = tables
        
        self.program_on_basic = ''
        self.array_interval = ''
        self.var_pool = []
        self.state = 'S'
    
    def value(self, token):
        if token[0] == 'C':
            return self.pascal_tables[token[0]][token[1]]['value']
        if token[0] == 'I':
            return self.pascal_tables[token[0]][token[1]]['name']
        return self.pascal_tables[token[0]][token[1]]
    
    def back(self, token):
        self.tokens_chain = [token] + self.tokens_chain
        
    
    def nextStep(self):
        try:
            token = self.tokens_chain[0]
            self.tokens_chain = self.tokens_chain[1:]
        except:
            token = []
            return self.program_on_basic
        
        if self.state == 'S':
            
            if self.value(token) == 'program':
                self.state = 'program'

            if self.value(token) == 'var':
                self.state = 'var'
                
            if self.value(token) == 'label':
                self.state = 'label'
                
            if self.value(token) == 'const':
                self.state = 'const'
                
            if self.value(token) == 'begin':
                pass
                
            return self.nextStep()
            
        if self.state == 'var':
            
            if token[0] == 'I':
                self.var_pool.append(self.value(token))
                
            if self.value(token) in ['integer', 'real', 'string']:
                self.var_pool = []
            
            if self.value(token) == '[':
                self.state = 'var ['
            
            if token[0] == 'W' and (not self.value(token) in ['array', 'of', 'integer', 'real', 'string']):
                self.state = 'S'
                self.back(token)
                
            return self.nextStep()
        
        if self.state == 'var [':
            
            if self.value(token) == ']':
                self.state = 'var []'
            else:
                self.array_interval += self.value(token)
            
            return self.nextStep()
        
        if self.state == 'var []':
            
            if self.value(token) in ['integer', 'real', 'string']:
                var_type = self.value(token)
                
                new_code = 'DIM '
                for var in self.var_pool:
                    new_code += var
                    if var_type == 'string':
                        new_code += '$'
                    if var_type == 'integer':
                        new_code += '%'
                    if var_type == 'real':
                        new_code += '!'
                        
                    new_code += '(' + self.array_interval.replace('..', ':').replace(',', ', ') + '), '
                new_code = new_code[:-2]
                
                self.array_interval = ''
                self.var_pool = []
                
                self.program_on_basic += new_code + '\n'
                print(new_code)
                self.state = 'var'
                
            return self.nextStep()         
        
        
        if self.state == 'const':
            if token[0] == 'W':
                self.back(token)
                self.state = 'S'
            return self.nextStep()
        
        if self.state in ['program', 'label']:
            if self.value(token) == ';':
                self.state = 'S'
            return self.nextStep()
            
            
        return self.program_on_basic
            

In [268]:
result = BasicGenerator(
    tokens_chain,
    {
        'W': ['program', 'var', 'const', 'integer', 'real', 'string', 'label',
                 'array', 'of', 'procedure', 'function',
                 'begin', ':=', 'goto', 'if', 'then', 'else', 'end', 'end.'],
        'O': ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>='],
        'R': [' ', ',', '..', ':', ';', '(', ')', '[', ']', '{', '}', '\''],
        'C': constants,
        'I': identifiers,
    }
)
output = result.nextStep()
print()
print(output)

DIM super_array%(1:3, 10:20)
DIM kk$(1:2)

DIM super_array%(1:3, 10:20)
DIM kk$(1:2)

